4 文本和字节序列

4.1 字符问题

一个字符串就是一个字符序列。

2015年，‘字符’最佳定义是unicode字符。因此，在python3str对象中获取的元素是unicode字符，这相当于从python2的unicode对象中获取的元素，而不是从python2的str对象中获取的原始字节序列。

字符的标识，即码位，是0~1114111的数字（十进制），在Unicode标准中以4~6个十六进制数字表示而且加前缀'U+'。例如，字母A的码位是U+0041，欧元符号的码位是U+20AC。在Unicode6.3中，约10%的有效码位有对应的字符。

字符的具体表述取决于所用的编码。编码是在码位和字节序列之间转换是使用的算法。在UTF-8编码中，A(U+0041)的码位编码成单个字节\x41,而在UTF-16LE中编码成两个字节\x41\x00.

把码位转换为字节序列的过程是编码；把字节序列转换为码位的过程是解码。

把字节序列变成人类可读的文本字符串就是解码，而把字符串变成用于存储或传输的字节序列就是编码。

In [1]:
s= 'café'
len(s)

4

In [2]:
b=s.encode('utf8')#使用utf-8把str对象编码成bytes对象。
b#bytes字面量以b开头

b'caf\xc3\xa9'

In [3]:
len(b)#在utf-8中，é的码位编码成两个字节

5

In [4]:
b.decode('utf8')

'café'

s[0]==s[:1]只对str这个序列类型成立，str类型的这个行为十分罕见。对其他各个序列类型来说，s[i]返回一个元素，s[i:i+1]返回一个相同类型的序列，里面是s[i]元素

4.2 字节概要

bytes或bytearray对象的各个元素是介于0~255的整数，而不像python2的str对象那样是单个的字符。然而，二进制序列的切片始终是同一类型的二进制序列，包括长度为1的切片。

In [5]:
cafe=bytes('café',encoding='utf8')#后面的两个字节\xc3\xa9对应é，前面的caf三个字节对应caf
cafe

b'caf\xc3\xa9'

In [6]:
cafe[0]#bytes或bytearray对象的各个元素是介于0~255的整数

99

In [7]:
cafe[1]

97

In [8]:
cafe[:1]#二进制序列的切片始终是同一类型的二进制序列，包括长度为1的切片。

b'c'

In [9]:
cafe[1:4]

b'af\xc3'

In [11]:
cafe[-1:]

b'\xa9'

In [12]:
cafe_arr = bytearray(cafe)
cafe_arr#bytearray对象没有字面量句法，而是以bytearray()和字节序列字面量参数的形式显示

bytearray(b'caf\xc3\xa9')

In [13]:
cafe_arr[:1]

bytearray(b'c')

In [14]:
cafe_arr[-1:]

bytearray(b'\xa9')

In [15]:
cafe_arr[-2:]

bytearray(b'\xc3\xa9')

In [16]:
cafe_arr[1]

97

s[0]==s[:1]只对str这个序列类型成立，str类型的这个行为十分罕见。对其他各个序列类型来说，s[i]返回一个元素，s[i:i+1]返回一个相同类型的序列，里面是s[i]元素

In [17]:
s='i am teacher'
s.replace('i','I',1)

'I am teacher'

In [18]:
s.endswith('er',2,12)

True

In [19]:
s.endswith('er',2,11)

False

In [20]:
cafe.endswith(b'\xc3\xa9')

True

处理格式化方法(format,format_map)和几个要处理unicode数据的方法（包括isdecimal,isnumeric...)str类型的其他方法都支持bytes和bytearray类型，意味着我们可以用endswith，replace，strip，translate，upper等方法处理二进制序列

In [21]:
cafe.upper()

b'CAF\xc3\xa9'

二进制有个独有的类方法fromhex，解析十六进制数字对构建二进制序列

In [27]:
bytes.fromhex('31 4B ce a9')

b'1K\xce\xa9'

bytes和bytearray实例还可以调用各自的构造方法，传入下列参数

一个str对象和一个encoding关键字参数

一个可迭代对象，提供0~255之间的数值

一个整数，使用空字节创建对应长度的二进制序列

In [23]:
bytes('wjy',encoding='utf8')

b'wjy'

In [24]:
lst=[1,2,3,4]
bytes(lst)

b'\x01\x02\x03\x04'

In [25]:
bytes(3)

b'\x00\x00\x00'

In [26]:
import array
numbers=array.array('h',[-2,-1,0,1,2,127,256])#256溢出了，-1：\ff\ff,-2:\fe\ff(后面的\ff代表负号，\fe=254,254+2=256
octsets=bytes(numbers)
octsets

b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00\x7f\x00\x00\x01'

4.4 了解编码问题

4.4.1 处理unicodeEncodeError

多数非utf编解码器只能处理unicode字符的一部分子集。把文本转换成字节序列时，如果目标编码没有定义某个字符，就会抛出UnicodeEncodeError异常，除非把errors参数传给编码方法或函数，对错误进行特殊处理。

In [33]:
b_city=bytes('S\xc3\xa3o Paulo',encoding='utf8')
b_city.decode('utf8')

'SÃ£o Paulo'

In [35]:
b'S\xc3\xa3o'.decode('utf8')

'São'

In [36]:
city='São'
city.encode('utf8')

b'S\xc3\xa3o'

In [37]:
city.encode('utf16')

b'\xff\xfeS\x00\xe3\x00o\x00'

In [38]:
city.encode('iso8859_1')

b'S\xe3o'

In [39]:
city.encode('cp437')

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [40]:
city.encode('cp437',errors='ignore')

b'So'

In [41]:
city.encode('cp437',errors='replace')

b'S?o'

In [42]:
city.encode('cp437',errors='xmlcharrefreplace')#把无法转换的字符替换成XML实体

b'S&#227;o'

4.4.2 处理UnicodeDecodeError

不是每一个字节都包含有效的ASCII字符，也不是没一个字符序列都是有效的utf-8或utf-16.因此，把二进制序列转换成文本时，如果假设是这两个编码中的一个，遇到无法转换的字节序列时会抛出UnicodeDecodeError。

另一方面，许多陈旧的8位编码，如'cp1252'等，本办法解码任何字节序列流而不抛出错误，例如随机噪声。因此，如果程序使用错误的8位编码，解码可能得到的是乱码，而不抛出错误。

In [44]:
octets = b'Montr\xe9al'
octets.decode('cp1252')#可以使用cp1252解码，他是latin1的有效超集

'Montréal'

In [48]:
octets.decode('iso8859_7')#ISO-8859-7用于编码希腊文，因此无法正确解释'\xe9'字节，而且没抛出错误

'Montrιal'

In [49]:
octets.decode('koi8_r')#KOI8-R用于编码俄文；这里'\xe9'表示西里尔字母И

'MontrИal'

In [50]:
octets.decode('utf8')#utf8编解码器检测到octets不是有效的utf-8字符串，抛出UnicodeDecodeError

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [51]:
octets.decode('utf8',errors='replace')

'Montr�al'

4.4.3 使用预期之外的编码加载模块时抛出的syntaxError

python3默认使用utf-8编码，python2则默认用ascii。如果加载的.py模块中包含utf-8之外的数据，而且没有声明编码，则会抛出SyntaxError

GNU/linux和OS X系统大多使用utf-8，因此打开在windows系统中使用cp1252编码.py文件时可能会抛出syntax error。

为了修正整个问题，可在文件顶部添加一行：#coding:cp1252

4.4.4 如何找出字节序列的编码

简单来说，不能，必须有人告诉你

4.4.5 BOM：有用的鬼符

In [59]:
b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'.decode('utf_16')

'El Niño'

In [60]:
u16='El Niño'.encode('utf16')

In [61]:
u16  

b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'

b'\xff\xfe'，这是BOM，即字节序标记(byte-order mark)，指明编码时使用Intel CPU的小字节序（小端）。

在小字节序设备中，各个码位的最低有效字节在前面：字母'E'的码位是U+0045(十进制数69），在字节偏移的第2位和第三位为69和0.正常应该为0069.

In [62]:
list(u16)

[255, 254, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

在大字节序CPU中，编码顺序是相反的；'E'编码为0和69

为了避免混淆，utf16在小字节序前加了b'\xff\xfe'(十进制数255,254）

In [64]:
u16le='El Niño'.encode('utf_16le')#le:little end

In [65]:
list(u16le)

[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]

In [66]:
u16be='El Niño'.encode('utf_16be')#be:big end

In [67]:
list(u16be)

[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]

4.5 处理文本文件

unicode三明治：尽早把输入（例如读文件时）的字节序列解码成字符串。编程过程处理字符串对象。对输出来说，近晚地把字符串编码成字节序列。

In [70]:
open('cafe.txt','w',encoding='utf8').write('café')

4

In [72]:
open('cafe.txt').read()#写入文件时指定了utf-8编码，但是读取文件时没有这么做，因此python假定使用windows系统默认编码（windows 1252）

'caf茅'

In [73]:
open('cafe.txt',encoding='utf8').read()

'café'

In [86]:
fp = open('cafe.txt','w',encoding='utf8')
fp

<_io.TextIOWrapper name='cafe.txt' mode='w' encoding='utf8'>

In [87]:
fp.write('café')

4

In [88]:
fp.close()

In [89]:
import os
os.stat('cafe.txt').st_size#utf8编码中é占两个字节,0xc3和0xa9

5

In [90]:
fp2=open('cafe.txt')

In [92]:
fp2#我的win10系统是cp936，前面的有些地方说错了

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='cp936'>

In [93]:
fp2.encoding

'cp936'

In [94]:
fp2.read()

'caf茅'

In [95]:
fp3=open('cafe.txt',encoding='utf8')

In [96]:
fp3

<_io.TextIOWrapper name='cafe.txt' mode='r' encoding='utf8'>

In [97]:
fp3.read()

'café'

In [98]:
fp4=open('cafe.txt','rb')#rb指明在二进制模式中读取文件
fp4

<_io.BufferedReader name='cafe.txt'>

编码默认值

In [99]:
import sys, locale

expressions = """
        locale.getpreferredencoding()
        type(my_file)
        my_file.encoding
        sys.stdout.isatty()
        sys.stdout.encoding
        sys.stdin.isatty()
        sys.stdin.encoding
        sys.stderr.isatty()
        sys.stderr.encoding
        sys.getdefaultencoding()
        sys.getfilesystemencoding()
    """

my_file = open('dummy', 'w')

for expression in expressions.split():
    value = eval(expression)
    print(expression.rjust(30), '->', repr(value))

 locale.getpreferredencoding() -> 'cp936'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'cp936'
           sys.stdout.isatty() -> False
           sys.stdout.encoding -> 'UTF-8'
            sys.stdin.isatty() -> False
            sys.stdin.encoding -> 'UTF8'
           sys.stderr.isatty() -> False
           sys.stderr.encoding -> 'UTF-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'mbcs'


关于编码默认值的最佳建议是：别依赖默认值。显式指明编解码类型。

4.6 为了正确比较而规范化的unicode字符串

In [101]:
s1='café'
s2='cafe\u0301'
s1,s2

('café', 'café')

In [103]:
len(s1),len(s2)

(4, 5)

In [104]:
s1 == s2

False

在unicode中，'é'和'e\u0301'这样的序列叫标准等价物，应用程序应该把它们视为相同字符，但是python看到的是不同的码位序列，因此判断不相等

这个问题的解决方案是使用unicodedata.normalize函数提供的unicode规范化。这个函数第一个参数为下列之一：NFC/NFD/NFKC/NFKD

NFC使用最少码位构成等价的字符串
NFD把组合字符分解为基字符和单独的组合字符。

In [105]:
from unicodedata import normalize
len(normalize('NFC',s1)),len(normalize('NFC',s2))

(4, 4)

In [106]:
len(normalize('NFD',s1)),len(normalize('NFD',s2))

(5, 5)

In [107]:
normalize('NFC',s1)==normalize('NFC',s2)

True

西方键盘通常能输出组合字符，因此用户输入的文本默认是NFC形式。不过，安全起见，保存文本之前，最好使用normalize('NFC',user_text)清洗字符串。

使用NFC时，有些单字符会被规范成另一单字符。如电阻的但闻欧姆会被规范成希腊字母大写的欧米茄。

In [108]:
from unicodedata import normalize,name
ohm='\u2126'
name(ohm)

'OHM SIGN'

In [109]:
ohm_c = normalize('NFC',ohm)
name(ohm_c)


'GREEK CAPITAL LETTER OMEGA'

In [111]:
ohm == ohm_c

False

4.6.1 大小写折叠

把所有文本变成小写。str.casefold()

对于只包含latin1字符的字符串s，s.casefold()得到的结果和s.lower()一样，唯有两个例外:微符号会变成小写的希腊字母，德语Eszett会变成ss。

4.6.2 规范化文本匹配实用函数

In [117]:
s1 = 'café'
s2 = 'cafe\u0301'
s1 == s2

False

In [121]:
from unicodedata import normalize

def nfc_equal(str1,str2):
    return normalize('NFC',str1)==normalize('NFC',str2)

def fold_equal(str1,str2):
    return (normalize('NFC',str1).casefold() == normalize('NFC',str2).casefold())

nfc_equal(s1,s2)

True

In [122]:
nfc_equal('A','a')

False

In [123]:
s3 = 'Straße'
s4 = 'Strasse'
s3 ==s4

False

In [124]:
nfc_equal(s3,s4)

False

In [126]:
fold_equal(s3,s4)

True

4.6.3 极端规范化：去掉变音符号

In [128]:
fold_equal('A','a')

True

In [129]:
import unicodedata
import string


def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # <2>
    return unicodedata.normalize('NFC', shaved)  # <3>
# END SHAVE_MARKS

# BEGIN SHAVE_MARKS_LATIN
def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>
# END SHAVE_MARKS_LATIN

# BEGIN ASCIIZE
single_map = str.maketrans("""‚ƒ„†ˆ‹‘’“”•–—˜›""",  # <1>
                           """'f"*^<''""---~>""")

multi_map = str.maketrans({  # <2>
    '€': '<euro>',
    '…': '...',
    'Œ': 'OE',
    '™': '(TM)',
    'œ': 'oe',
    '‰': '<per mille>',
    '‡': '**',
})

multi_map.update(single_map)  # <3>


def dewinize(txt):
    """Replace Win1252 symbols with ASCII chars or sequences"""
    return txt.translate(multi_map)  # <4>


def asciize(txt):
    no_marks = shave_marks_latin(dewinize(txt))     # <5>
    no_marks = no_marks.replace('ß', 'ss')          # <6>
    return unicodedata.normalize('NFKC', no_marks)  # <7>
# END ASCIIZE

In [129]:
 order = '“Herr Voß: • ½ cup of Œtker™ caffè latte • bowl of açaí.”'

In [131]:
shave_marks(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [132]:
shave_marks_latin(order)

'“Herr Voß: • ½ cup of Œtker™ caffe latte • bowl of acai.”'

In [133]:
dewinize(order)

'"Herr Voß: - ½ cup of OEtker(TM) caffè latte - bowl of açaí."'

In [134]:
 asciize(order)

'"Herr Voss: - 1⁄2 cup of OEtker(TM) caffe latte - bowl of acai."'

4.7 unicode文本排序

python比较任何类型的序列时，会注意比较序列的各个元素。对字符串来说，比较的是码位。但在比较非ascii字符时，得到结果不如愿

In [139]:
fruit=['caju','atemoia','caja\u0301','ac\u0301ai\u0301','acerola']
sorted(fruit)

['acerola', 'aćaí', 'atemoia', 'cajá', 'caju']

python中，非ASCII文本的标准排序方式是使用locale.strxfrm函数。

In [140]:
import locale
locale.setlocale(locale.LC_COLLATE,'pt_BR.UTF-8')#windows不支持locale设置

Error: unsupported locale setting

使用unicode排序算法排序

In [141]:
import pyuca
coll = pyuca.Collator()
sorted_fruit=sorted(fruit,key=coll.sort_key)
sorted_fruit

['aćaí', 'acerola', 'atemoia', 'cajá', 'caju']

4.8 unicode数据库

In [142]:
import unicodedata
import re
re_digit=re.compile(r'\d')
sample='1\xbc\xb2\u0969\u136b\u216b\u2466\u2480\u3285'

for char in sample:
    print(
        'U+%04x' % ord(char),
        char.center(6),
        're_dig' if re_digit.match(char) else '-',#re不能匹配该例中的许多数字，re模块对unicode的支持并不充分
        'isdig' if char.isdigit() else '-',
        'isnum' if char.isnumeric() else '-',
        format(unicodedata.numeric(char),'5.2f'),#长度为5，小数点2位
        unicodedata.name(char),#unicode标准中字符名字
        sep='\t'
    )

U+0031	  1   	re_dig	isdig	isnum	 1.00	DIGIT ONE
U+00bc	  ¼   	-	-	isnum	 0.25	VULGAR FRACTION ONE QUARTER
U+00b2	  ²   	-	isdig	isnum	 2.00	SUPERSCRIPT TWO
U+0969	  ३   	re_dig	isdig	isnum	 3.00	DEVANAGARI DIGIT THREE
U+136b	  ፫   	-	isdig	isnum	 3.00	ETHIOPIC DIGIT THREE
U+216b	  Ⅻ   	-	-	isnum	12.00	ROMAN NUMERAL TWELVE
U+2466	  ⑦   	-	isdig	isnum	 7.00	CIRCLED DIGIT SEVEN
U+2480	  ⒀   	-	-	isnum	13.00	PARENTHESIZED NUMBER THIRTEEN
U+3285	  ㊅   	-	-	isnum	 6.00	CIRCLED IDEOGRAPH SIX


4.9 支持字符串和字节序列的双模式API

标准库中的一些函数能接受字符串或字节序列为参数，然后根据类型展现不同的行为。re和os模块中就有这样的函数。

4.9.1正则表达式中字符串和字节序列

In [143]:
import re

re_numbers_str = re.compile(r'\d+')     # <1>字符串类型
re_words_str = re.compile(r'\w+')
re_numbers_bytes = re.compile(rb'\d+')  # <2>字节序列类型
re_words_bytes = re.compile(rb'\w+')

text_str = ("Ramanujan saw \u0be7\u0bed\u0be8\u0bef"  # <3>
            " as 1729 = 1³ + 12³ = 9³ + 10³.")        # <4>

text_bytes = text_str.encode('utf_8')  # <5>#字节序列只能用字节序列正则表达式来搜索

print('Text', repr(text_str), sep='\n  ')
print('Numbers')
print('  str  :', re_numbers_str.findall(text_str))      # <6>字符串模式能匹配泰米尔数字和ASCII数字
print('  bytes:', re_numbers_bytes.findall(text_bytes))  # <7>字节序列模式只能匹配ASCII字节中的数字
print('Words')
print('  str  :', re_words_str.findall(text_str))        # <8>字符串模式能匹配字母、上标、泰米尔数字和ASCII数字
print('  bytes:', re_words_bytes.findall(text_bytes))    # <9>字节序列模式只能匹配ASCII字节中的字母和数字
# END RE_DEMO

Text
  'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
  str  : ['௧௭௨௯', '1729', '1', '12', '9', '10']
  bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
  str  : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
  bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


4.9.2 os函数中的字符串和字节序列

![chapter4](image/chapter4.jpg)

In [144]:
import os
os.listdir('.')

['.ipynb_checkpoints',
 'add2virtualenv.bat',
 'automat-visualize.exe',
 'cafe.txt',
 'cd-.bat',
 'cdproject.bat',
 'cdsitepackages.bat',
 'cdvirtualenv.bat',
 'cftp.exe',
 'chardetect.exe',
 'ckeygen.exe',
 'conch.exe',
 'csv2ods',
 'cygdb.exe',
 'cython.exe',
 'cythonize.exe',
 'django-admin.exe',
 'django-admin.py',
 'dummy',
 'easy_install-3.5.exe',
 'easy_install.exe',
 'epylint.exe',
 'f2py.py',
 'flake8.exe',
 'flask.exe',
 'floats.bin',
 'folder_delete.bat',
 'futurize.exe',
 'iptest.exe',
 'iptest3.exe',
 'ipython.exe',
 'ipython3.exe',
 'isort.exe',
 'isympy',
 'jsonschema.exe',
 'jupyter-bundlerextension.exe',
 'jupyter-console.exe',
 'jupyter-kernel.exe',
 'jupyter-kernelspec.exe',
 'jupyter-migrate.exe',
 'jupyter-nbconvert.exe',
 'jupyter-nbextension.exe',
 'jupyter-notebook.exe',
 'jupyter-notebook.exe.lnk',
 'jupyter-qtconsole.exe',
 'jupyter-run.exe',
 'jupyter-serverextension.exe',
 'jupyter-troubleshoot.exe',
 'jupyter-trust.exe',
 'jupyter.exe',
 'lssitepackages.bat

In [145]:
pwd

'C:\\Program Files\\Python35\\Scripts'

In [146]:
os.listdir(b'.')

c:\program files\python35\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The Windows bytes API has been deprecated, use Unicode filenames instead
  """Entry point for launching an IPython kernel.


[b'.ipynb_checkpoints',
 b'add2virtualenv.bat',
 b'automat-visualize.exe',
 b'cafe.txt',
 b'cd-.bat',
 b'cdproject.bat',
 b'cdsitepackages.bat',
 b'cdvirtualenv.bat',
 b'cftp.exe',
 b'chardetect.exe',
 b'ckeygen.exe',
 b'conch.exe',
 b'csv2ods',
 b'cygdb.exe',
 b'cython.exe',
 b'cythonize.exe',
 b'django-admin.exe',
 b'django-admin.py',
 b'dummy',
 b'easy_install-3.5.exe',
 b'easy_install.exe',
 b'epylint.exe',
 b'f2py.py',
 b'flake8.exe',
 b'flask.exe',
 b'floats.bin',
 b'folder_delete.bat',
 b'futurize.exe',
 b'iptest.exe',
 b'iptest3.exe',
 b'ipython.exe',
 b'ipython3.exe',
 b'isort.exe',
 b'isympy',
 b'jsonschema.exe',
 b'jupyter-bundlerextension.exe',
 b'jupyter-console.exe',
 b'jupyter-kernel.exe',
 b'jupyter-kernelspec.exe',
 b'jupyter-migrate.exe',
 b'jupyter-nbconvert.exe',
 b'jupyter-nbextension.exe',
 b'jupyter-notebook.exe',
 b'jupyter-notebook.exe.lnk',
 b'jupyter-qtconsole.exe',
 b'jupyter-run.exe',
 b'jupyter-serverextension.exe',
 b'jupyter-troubleshoot.exe',
 b'jupyter